In [ ]:
import numpy as nmp
import pandas as pd
import random
import matplotlib.pyplot as pltIm
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow import keras

# Downloading Dataset

In [ ]:
(Xtrain,Ytrain),(Xtest,Ytest) = tf.keras.datasets.cifar10.load_data()

In [ ]:
for index in range(12):
    pltIm.subplot(3,4,index+1)
    pltIm.imshow(Xtrain[index])
pltIm.show()

# Normalize Pixel Data and Change Data Type

In [ ]:
def normalizePixelDataAndChangeType(train,test):
    normalizeAndTypeTrain = train.astype('float32')
    normalizeAndTypeTest = test.astype('float32')
    
    normalizeAndTypeTrain = normalizeAndTypeTrain / 255.0 - 0.5
    normalizeAndTypeTest = normalizeAndTypeTest / 255.0 - 0.5
    
    return normalizeAndTypeTrain , normalizeAndTypeTest

# Convolutional layer Class

In [ ]:
class convolutionLayerOperation:
    def __init__(self,numberOfFilters, sizeOfFilter):
        self.numberOfFilters = numberOfFilters
        self.sizeOfFilter = sizeOfFilter
        self.convolutionFilter = nmp.random.randn(self.numberOfFilters,self.sizeOfFilter,self.sizeOfFilter) / (self.sizeOfFilter * self.sizeOfFilter)
    def imagePatch(self):
        for vIndex in range(self.heightOfInput - self.sizeOfFilter + 1):
            for hIndex in range(self.widthOfInput - self.sizeOfFilter + 1):
                    imagePatch = self.imageInput[vIndex : (vIndex + self.sizeOfFilter) , hIndex : (hIndex + self.sizeOfFilter) ]
                    yield imagePatch,vIndex,hIndex
    
    def forwardPropagation(self,imageInput):
        self.imageInput = imageInput
        self.heightOfInput , self.widthOfInput = self.imageInput.shape
        convolutionOutput = nmp.zeros((self.heightOfInput - self.sizeOfFilter + 1, self.widthOfInput - self.sizeOfFilter + 1, self.numberOfFilters))
        
        for imagePatch , vIndex , hIndex in self.imagePatch():
            for filter_ in range(self.numberOfFilters):
                convolutionOutput[vIndex,hIndex] = nmp.sum(imagePatch * self.convolutionFilter,axis = (1,2))
        return convolutionOutput
    
    def backPropagation(self,dLossOutPrev,learningRate):
        dLossParams = nmp.zeros(self.convolutionFilter.shape)
        for imagePatch , vIndex,hIndex in self.imagePatch():
            for itrFilter in range(self.numberOfFilters):
                dLossParams[itrFilter] += imagePatch * dLossOutPrev[vIndex,hIndex,itrFilter]
        
        self.convolutionFilter = self.convolutionFilter - learningRate * dLossParams
        return dLossParams

# Pooling Class

In [ ]:
class pooling:
    def __init__(self,sizeOfFilter,typePooling):
        self.sizeOfFilter = sizeOfFilter
        self.typePooling = typePooling
        

    def calculateNewDimensionAndImagePatch(self):
        
        for vIndex in range(self.updatedHeight):
            for hIndex in range(self.updatedWidth):
                imagePatch = self.imageInput[vIndex * self.sizeOfFilter : vIndex * self.sizeOfFilter + self.sizeOfFilter , hIndex * self.sizeOfFilter : hIndex * self.sizeOfFilter + self.sizeOfFilter]
                yield imagePatch,vIndex,hIndex
    
    def poolingOperation(self):
        
        poolOutput = nmp.zeros((self.updatedHeight,self.updatedWidth,self.numberOfFilters))
        
        if self.typePooling == "Max":
            for imagePatch , vIndex, hIndex in self.calculateNewDimensionAndImagePatch():
                poolOutput[vIndex,hIndex] = nmp.amax(imagePatch,axis = (0,1))
        else:
            for imagePatch , vIndex, hIndex in self.calculateNewDimensionAndImagePatch():
                poolOutput[vIndex,hIndex] = nmp.mean(imagePatch,axis = (0,1))
        
        return poolOutput
    
    def forwardPropagation(self,imageInput):
        self.imageInput = imageInput
        self.heightInput , self.widthInput , self.numberOfFilters = self.imageInput.shape[0], self.imageInput.shape[1],self.imageInput.shape[2]
        self.updatedHeight = self.heightInput // self.sizeOfFilter
        self.updatedWidth = self.widthInput // self.sizeOfFilter
        
        poolOutput = self.poolingOperation()
        
        return poolOutput
    
    def backPropagation(self,dLossOutPrev):
        
        dLossMaxPool = nmp.zeros(self.imageInput.shape)
        
        for imagePatch , vIndex, hIndex in self.calculateNewDimensionAndImagePatch():
            heightImagePatch , widthImagePatch , numberOfFilters = imagePatch.shape
            maxVal = nmp.amax(imagePatch,axis = (0,1))
            for vIndex_ in range(heightImagePatch):
                for hIndex_ in range(widthImagePatch):
                    for filter_ in range(numberOfFilters):
                        if imagePatch[vIndex_,hIndex_,filter_] == maxVal[filter_]:
                            dLossMaxPool[vIndex * self.sizeOfFilter + vIndex_ , hIndex * self.sizeOfFilter + hIndex_,filter_] = dLossOutPrev[vIndex,hIndex,filter_]
                    
            return dLossMaxPool

# Softmax Class

In [ ]:
class softmax:
    def __init__(self,inputSoftmax,softmaxNode):
        self.weightsLayer = nmp.random.randn(inputSoftmax,softmaxNode)/inputSoftmax
        self.bias = nmp.zeros(softmaxNode)
    
    def forwardPropagation(self,imageInput):
        self.imageInput = imageInput
        self.inputShape = self.imageInput.shape
        
        self.flattenImage = self.imageInput.flatten()
        self.outputSoftmaxValue = nmp.dot(self.flattenImage,self.weightsLayer) + self.bias
        self.outputSoftmaxValue = nmp.exp(self.outputSoftmaxValue)
        return self.outputSoftmaxValue / nmp.sum(self.outputSoftmaxValue,axis = 0)
    
    def backPropagation(self,dLOut,learningRate):
        for index,grad in enumerate(dLOut):
            if grad == 0:
                continue
            
            totalSum = nmp.sum(self.outputSoftmaxValue,axis = 0)
            
            dydz = - self.outputSoftmaxValue[index] * self.outputSoftmaxValue / (totalSum ** 2)
            dydz[index] = self.outputSoftmaxValue[index]*(totalSum - self.outputSoftmaxValue[index]) / (totalSum ** 2)
            
            dzdw = self.flattenImage
            dzdb = 1
            dzdinput = self.weightsLayer
            
            dLdz = grad * dydz
            
            dLdw = dzdw[nmp.newaxis].T @ dLdz[nmp.newaxis]
            dLdb = dLdz * dzdb
            dLdinput = dzdinput @ dLdz
        
        
        self.weightsLayer = self.weightsLayer - learningRate * dLdw
        self.bias = self.bias - learningRate * dLdb
        
        return dLdinput.reshape(self.imageInput.shape)

# Fully Connected Layer Class

In [ ]:
class fullyConnectedLayer:
    
    def __init__(self,inputSize,outputSize):
    
        self.weightsLayer = nmp.random.randn(inputSize,outputSize) / (inputSize * outputSize)

        self.bias = nmp.random.rand(1, outputSize)
        
    
    def forwardPropagation(self,inputData):
        self.inputData = inputData
        self.output = nmp.dot(self.inputData,self.weightsLayer) + self.bias
        return self.output
    
    def backPropagation(self,outputError,learningRate):
        inputError = nmp.dot(outputError,self.weightsLayer.T)
        
        weightsError = nmp.dot(self.inputData.T,outputError)
        
        self.weightsLayer = self.weightsLayer - learningRate * weightsError
        self.bias = self.bias - learningRate * outputError
        return inputError

# Activation Layer Class

In [ ]:
class ActivationLayer():
    def __init__(self,typeAct = 'ReLU'):
        self.typeAct = typeAct
    
    def forwardPropagation(self,inputData):
        self.inputActivation = inputData
        self.output = self.activationFunction(self.inputActivation)
        return self.output
    
    def activationFunction(self,inputMatrix):
        if self.typeAct == 'sigmoid':
            return 1 / (1 + nmp.exp(-inputMatrix))
        elif self.typeAct == 'tanh':
            return nmp.tanh(inputMatrix)
        else:
            inputMatrix[inputMatrix < 0] = 0
            return inputMatrix
        
    def firstOrderDerivative(self,inputMatrix):
        if self.typeAct == 'sigmoid':
            return inputMatrix * (1 - inputMatrix)
        elif self.typeAct == 'tanh':
            return 1 - inputMatrix ** 2
        else:
            inputMatrix[inputMatrix < 0] = 0
            inputMatrix[inputMatrix >= 1] = 1
            return inputMatrix
        
    def backPropagation(self,outputError,learningRate):
        
        outputD = self.firstOrderDerivative(self.inputActivation) * outputError
        return outputD

# Flatten Layer Class

In [ ]:
class flattenLayer():
    def __init__(self):
        pass
    def forwardPropagation(self, inputData):
        self.input = inputData
        self.output = inputData.flatten().reshape((1,-1))
        return self.output
    
    def backPropagation(self, outputError, learningRate):
        return outputError.reshape(self.input.shape)

In [ ]:
conv1 = convolutionLayerOperation(20,3)
ACLayer1 = ActivationLayer('ReLU')
mPool1 = pooling(2,'Max')

flattenL1 = flattenLayer()
FC1 = fullyConnectedLayer(15*15*20,100)
ACLayer2 = ActivationLayer('ReLU')
softmax1 = softmax(100,10)

In [ ]:
def convolutionForwardPropagation(image,conv,aCLayer,pool):
    
    output = conv.forwardPropagation(image)
    output = aCLayer.forwardPropagation(output)
    output = pool.forwardPropagation(output)  
    return output

In [ ]:
def calculateLossAndAccuracy(output,lbl):
    crossEntropyLoss = -nmp.log(output[lbl])
    accuracyEvaluation = 1 if nmp.argmax(output) == lbl else 0
    
    return crossEntropyLoss, accuracyEvaluation

In [ ]:
def trainingCNN(image,lbl,learningRate = 0.05):
    
    #Forward
    output = convolutionForwardPropagation(image,conv1,ACLayer1,mPool1)
    output = flattenL1.forwardPropagation(output)
    output = FC1.forwardPropagation(output)
    output = ACLayer2.forwardPropagation(output)
    output = softmax1.forwardPropagation(output)
    
    loss, accuracy = calculateLossAndAccuracy(output,lbl)
    
    #Calculate Gradient
    gradient = nmp.zeros(10)
    gradient[lbl] = -1/ output[lbl]
    
    gradBack = softmax1.backPropagation(gradient,learningRate)
    gradBack = ACLayer2.backPropagation(gradBack,learningRate)
    gradBack = FC1.backPropagation(gradBack,learningRate)
    gradBack = flattenL1.backPropagation(gradBack,learningRate)
    
    gradBack = mPool1.backPropagation(gradBack)
    gradBack = ACLayer1.backPropagation(gradBack,learningRate)
    gradBack = conv1.backPropagation(gradBack,learningRate)
    
    return loss,accuracy

In [ ]:
def trainingLoop(epoch,Xtrain,Ytrain,learningRate):
    for epoch_ in range(epoch):
        print(f"Epoch : {epoch_ + 1}")
        loss = 0
        numCorr = 0

        tempData = list(zip(Xtrain,Ytrain))
        random.shuffle(tempData)
        Xtrain , Ytrain = zip(*tempData)
        Xtrain,Ytrain = list(Xtrain),list(Ytrain)
    
        for index , (image,lbl) in enumerate(zip(Xtrain,Ytrain)):
        
            if index % 125 == 0:
                print(f"{index+1} steps out of 100:Average Loss {loss/100} and Accuracy {numCorr/125 * 100}%")
                loss = 0
                numCorr = 0
        
            loss_ , accu = trainingCNN(image,lbl,learningRate)
            loss += loss_
            numCorr += accu

In [ ]:
XtrainG = []
XtestG = []

In [ ]:
def convertToGray(imageData,type_ = "train"):
    
    for imageIn in imageData:
        redComponent, gComponent, bComponent = imageIn[:,:,0], imageIn[:,:,1], imageIn[:,:,2]
        imageIn = 0.2989 * redComponent + 0.5870 * gComponent + 0.1140 * bComponent
        
        if(type_ == "train"):
            XtrainG.append(imageIn)
        else:
            XtestG.append(imageIn)

In [ ]:
XtrainTL , XtestTL = normalizePixelDataAndChangeType(Xtrain,Xtest)

In [ ]:
convertToGray(XtrainTL,"train")
convertToGray(XtestTL,"test")
XtrainG = XtrainG[:10000]
YtrainG = Ytrain[:10000]
XtestG = XtestG[:10000]
YtestG = Ytest[:10000]

In [ ]:
len(XtrainG)

In [ ]:
XtrainG[0].shape

In [ ]:
trainingLoop(10,XtrainG,YtrainG,0.05)

# Transfer Learning

In [ ]:
from tensorflow import keras
from keras.applications.vgg19 import VGG19

In [ ]:
imageSize = [32,32]

In [ ]:
vggModel = VGG19(input_shape = imageSize + [3] ,weights = 'imagenet',include_top = False)

In [ ]:
vggModel.summary()

In [ ]:
for individualLayer in vggModel.layers:
    individualLayer.trainable = False

In [ ]:
layersX = tf.keras.layers.Flatten()(vggModel.output)
predicOutputOf = tf.keras.layers.Dense(10,activation = 'softmax')(layersX)
changedPretainedModel = tf.keras.models.Model(inputs = vggModel.input,outputs = predicOutputOf)

In [ ]:
optimizerUsedIntheModel = keras.optimizers.Adam(learning_rate = 0.05)
metricsUsedInTheModel = ["accuracy"]
changedPretainedModel.compile(optimizer = optimizerUsedIntheModel ,loss = 'sparse_categorical_crossentropy',metrics = metricsUsedInTheModel)

In [ ]:
changedPretainedModel.fit(XtrainTL,Ytrain,validation_data = (XtestTL,Ytest),epochs = 10)